##  <font color = "navy">WEATHER FORECAST IN AUSTRALIA: Modélisation deep learning</font>
Notebook <b>3/3</b>
***

<font color = "purple"><b>Objectif: Réaliser un modèle de deep learning capable de détecter les jours de pluie à J+1 avec une précision satisfaisante.</b></font>

<b> Résumé du notebook : </b>
- Data cleaning, preprocessing des données (notamment encodage, et rééquilibrage) ;
- Modélisation deep learning ;
- Essai d'amélioration des performances du modèle.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import warnings
from sklearn.preprocessing import LabelEncoder
from IPython.display import Markdown, display
warnings.filterwarnings('ignore')
sns.set_theme({'legend.frameon':True})
from tensorflow.keras.layers import Input, Dense 
from tensorflow.keras.models import Model
import itertools 
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import ensemble
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

#pip install --upgrade tensorflow
#pip install pydot
#pip install graphviz

In [2]:
#Import d'une version prête à l'emploi (preprocessing: Cf notebook 2/4)
data = pd.read_csv("weatherAUS_DL.csv")

#Création d'un df de secours pour avoir une trace des données brutes.

df_saved = pd.read_csv("weatherAUS_DL.csv")

pd.set_option("display.max_columns", None)

feats = data.drop(["RainTomorrow", "Date", "Year", "Month", "Day", "State", "Location"], axis = 1)
target = data["RainTomorrow"]
encoder = LabelEncoder()

Y = encoder.fit_transform(target)

X_train, X_test, y_train, y_test = train_test_split(feats, Y, test_size = 0.2, random_state = 55) 

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [3]:
display(Markdown("""<h2><u><center>Réalisation d'un premier modèle de deep learning</center></u></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))
EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)
print(score)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce premier modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li>Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("""Les paramètres de ce premier modèle ont été fixés de façon plus ou moins arbritraire. Les 
performances de ce modèle sont globalement satisfaisantes, voire très satisfaisantes pour la détection des jours sans pluie.
Il serait toutefois intéressant d'améliorer la prédiction de jours de pluie."""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><u><center>Réalisation d'un premier modèle de deep learning</center></u></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 951us/step - loss: 0.3541 - accuracy: 0.8466 - val_loss: 0.3393 - val_accuracy: 0.8547
Epoch 2/6
3229/3229 [==============================] - 3s 909us/step - loss: 0.3446 - accuracy: 0.8506 - val_loss: 0.3365 - val_accuracy: 0.8546
Epoch 3/6
3229/3229 [==============================] - 3s 908us/step - loss: 0.3415 - accuracy: 0.8517 - val_loss: 0.3373 - val_accuracy: 0.8553
Epoch 4/6
3229/3229 [==============================] - 3s 900us/step - loss: 0.3397 - accuracy: 0.8530 - val_loss: 0.3353 - val_accuracy: 0.8551
Epoch 5/6
3229/3229 [==============================] - 3s 994us/step - loss: 0.3384 - accuracy: 0.8525 - val_loss: 0.3329 - val_accuracy: 0.8558
Epoch 6/6
808/808 [==============================] - 1s 667us/step - loss: 0.3326 - accuracy: 0.8568
[0.33264657855033875, 0.856772243976593]


<h4><u>Récapitulatif du modèle :</u></h4>
Ce premier modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li>Prédiction des <b>jours de pluie</b> avec une précision de <b>73.86 %</b> (<i>recall = <b>52.1 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.8 %</b> (<i>recall = <b>94.92 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.68 %</b>.</li></ul>


Les paramètres de ce premier modèle ont été fixés de façon plus ou moins arbritraire. Les 
performances de ce modèle sont globalement satisfaisantes, voire très satisfaisantes pour la détection des jours sans pluie.
Il serait toutefois intéressant d'améliorer la prédiction de jours de pluie.

<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.95      0.91     20250
           1       0.74      0.52      0.61      5576

    accuracy                           0.86     25826
   macro avg       0.81      0.74      0.76     25826
weighted avg       0.85      0.86      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19222  1028
1                2671  2905


In [4]:
display(Markdown("""<h2><center><u>Effet du nombre de neurones</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 250
UNITS2 = 500
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")


x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))
display(Markdown("""Le simple ajout de neurones (ici 10 fois plus nombreux dans chaque couche par rapport au modèle initial) 
ne semble pas modifier les performances."""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet du nombre de neurones</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 6s 2ms/step - loss: 0.3498 - accuracy: 0.8488 - val_loss: 0.3359 - val_accuracy: 0.8551
Epoch 2/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3407 - accuracy: 0.8521 - val_loss: 0.3339 - val_accuracy: 0.8565
Epoch 3/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3370 - accuracy: 0.8535 - val_loss: 0.3333 - val_accuracy: 0.8562
Epoch 4/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3338 - accuracy: 0.8548 - val_loss: 0.3312 - val_accuracy: 0.8566
Epoch 5/6
3229/3229 [==============================] - 5s 2ms/step - loss: 0.3318 - accuracy: 0.8556 - val_loss: 0.3303 - val_accuracy: 0.8571
Epoch 6/6
808/808 [==============================] - 1s 866us/step - loss: 0.3298 - accuracy: 0.8581


<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>250 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>500 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 250)               5750      
                                                                 
 Dense_2 (Dense)             (None, 500)               125500    
                                                                 
 Dense_3 (Dense)             (None, 2)                 1002      
                                                                 
Total params: 132,252
Trainable params: 132,252
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>73.0 %</b> (<i>recall = <b>54.39 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>88.27 %</b> (<i>recall = <b>94.46 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.81 %</b>.</li></ul>


Le simple ajout de neurones (ici 10 fois plus nombreux dans chaque couche par rapport au modèle initial) 
ne semble pas modifier les performances.

<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     20250
           1       0.73      0.54      0.62      5576

    accuracy                           0.86     25826
   macro avg       0.81      0.74      0.77     25826
weighted avg       0.85      0.86      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19128  1122
1                2543  3033


In [5]:
display(Markdown("""<h2><center><u>Effet du nombre d'épochs</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 18
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))
display(Markdown("""L'entrainement sur un nombre plus important d'epochs (ici 18 soit 3 fois plus par rapport au modèle initial) 
ne semble pas modifier les performances."""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet du nombre d'épochs</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/18
3229/3229 [==============================] - 3s 965us/step - loss: 0.3531 - accuracy: 0.8471 - val_loss: 0.3376 - val_accuracy: 0.8560
Epoch 2/18
3229/3229 [==============================] - 3s 918us/step - loss: 0.3433 - accuracy: 0.8504 - val_loss: 0.3382 - val_accuracy: 0.8536
Epoch 3/18
3229/3229 [==============================] - 3s 940us/step - loss: 0.3412 - accuracy: 0.8520 - val_loss: 0.3351 - val_accuracy: 0.8556
Epoch 4/18
3229/3229 [==============================] - 3s 922us/step - loss: 0.3391 - accuracy: 0.8524 - val_loss: 0.3353 - val_accuracy: 0.8558
Epoch 5/18
3229/3229 [==============================] - 3s 979us/step - loss: 0.3381 - accuracy: 0.8530 - val_loss: 0.3340 - val_accuracy: 0.8554
Epoch 6/18
3229/3229 [==============================] - 3s 916us/step - loss: 0.3371 - accuracy: 0.8545 - val_loss: 0.3327 - val_accuracy: 0.8555
Epoch 7/18
3229/3229 [==============================] - 3s 864us/step - loss: 0.3362 - accuracy: 0.8540 - val_loss: 0.3330 -

<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>18 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>72.23 %</b> (<i>recall = <b>53.35 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>88.02 %</b> (<i>recall = <b>94.35 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.5 %</b>.</li></ul>


L'entrainement sur un nombre plus important d'epochs (ici 18 soit 3 fois plus par rapport au modèle initial) 
ne semble pas modifier les performances.

<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     20250
           1       0.72      0.53      0.61      5576

    accuracy                           0.85     25826
   macro avg       0.80      0.74      0.76     25826
weighted avg       0.85      0.85      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19106  1144
1                2601  2975


In [24]:
display(Markdown("""<h2><center><u>Effet de la taille des batchs</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 320

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None

inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)

model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])

training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()
                 
display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("""L'entrainement sur des batchs de plus grande taille (ici 320 soit 10 fois plus grands par rapport au modèle
initial) ne semble pas modifier les performances, en revanche la vitesse d'execution du modèle est considérablement réduite.
"""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet de la taille des batchs</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
323/323 [==============================] - 1s 2ms/step - loss: 0.3850 - accuracy: 0.8315 - val_loss: 0.3430 - val_accuracy: 0.8528
Epoch 2/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3468 - accuracy: 0.8500 - val_loss: 0.3403 - val_accuracy: 0.8536
Epoch 3/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3449 - accuracy: 0.8503 - val_loss: 0.3388 - val_accuracy: 0.8534
Epoch 4/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3439 - accuracy: 0.8507 - val_loss: 0.3400 - val_accuracy: 0.8536
Epoch 5/6
323/323 [==============================] - 0s 1ms/step - loss: 0.3426 - accuracy: 0.8510 - val_loss: 0.3378 - val_accuracy: 0.8558
Epoch 6/6
808/808 [==============================] - 1s 610us/step - loss: 0.3383 - accuracy: 0.8542


<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>320</b>.</li></ul>


Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>72.24 %</b> (<i>recall = <b>52.73 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.88 %</b> (<i>recall = <b>94.42 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.42 %</b>.</li></ul>


L'entrainement sur des batchs de plus grande taille (ici 320 soit 10 fois plus grands par rapport au modèle
initial) ne semble pas modifier les performances, en revanche la vitesse d'execution du modèle est considérablement réduite.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     20250
           1       0.72      0.53      0.61      5576

    accuracy                           0.85     25826
   macro avg       0.80      0.74      0.76     25826
weighted avg       0.85      0.85      0.85     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19120  1130
1                2636  2940


In [25]:
display(Markdown("""<h2><center><u>Effet des fonctions d'activation</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = None
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "tanh"
ACTIV2 = "tanh"
ACTIV3 = None
ACTIV4 = None
ACTIV5 = None


inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_3")

x = dense1(inputs)
x = dense2(x)
outputs = dense3(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("""L'activation des couches denses par la fonction tangente hyperbolique (au lieu de relu dans modèle initial) 
ne semble pas modifier les performances."""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet des fonctions d'activation</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 909us/step - loss: 0.3593 - accuracy: 0.8444 - val_loss: 0.3494 - val_accuracy: 0.8514
Epoch 2/6
3229/3229 [==============================] - 3s 875us/step - loss: 0.3540 - accuracy: 0.8463 - val_loss: 0.3460 - val_accuracy: 0.8508
Epoch 3/6
3229/3229 [==============================] - 3s 898us/step - loss: 0.3508 - accuracy: 0.8483 - val_loss: 0.3439 - val_accuracy: 0.8501
Epoch 4/6
3229/3229 [==============================] - 3s 869us/step - loss: 0.3481 - accuracy: 0.8486 - val_loss: 0.3415 - val_accuracy: 0.8537
Epoch 5/6
3229/3229 [==============================] - 3s 876us/step - loss: 0.3469 - accuracy: 0.8485 - val_loss: 0.3419 - val_accuracy: 0.8532
Epoch 6/6
808/808 [==============================] - 0s 589us/step - loss: 0.3410 - accuracy: 0.8533


<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 2 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>tanh</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>tanh</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 2)                 102       
                                                                 
Total params: 1,977
Trainable params: 1,977
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>71.67 %</b> (<i>recall = <b>52.99 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.92 %</b> (<i>recall = <b>94.23 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.33 %</b>.</li></ul>


L'activation des couches denses par la fonction tangente hyperbolique (au lieu de relu dans modèle initial) 
ne semble pas modifier les performances.

<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.88      0.94      0.91     20250
           1       0.72      0.53      0.61      5576

    accuracy                           0.85     25826
   macro avg       0.80      0.74      0.76     25826
weighted avg       0.84      0.85      0.84     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19082  1168
1                2621  2955


In [26]:
display(Markdown("""<h2><center><u>Effet du nombre de couches de neurones</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>"""))

EPOCHS = 6
BATCHS = 32

UNITS1 = 25
UNITS2 = 50
UNITS3 = 50
UNITS4 = None
UNITS5 = None
UNITSOUT = 2

ACTIV1 = "relu"
ACTIV2 = "relu"
ACTIV3 = "relu"
ACTIV4 = None
ACTIV5 = None


inputs = Input(shape = X_train_scaled.shape[1], name = "Input")
dense1 = Dense(units = UNITS1, activation = ACTIV1, kernel_initializer = "normal", name = "Dense_1")
dense2 = Dense(units = UNITS2, activation = ACTIV2, kernel_initializer = "normal", name = "Dense_2")
dense3 = Dense(units = UNITS3, activation = ACTIV3, kernel_initializer = "normal", name = "Dense_3")
dense4 = Dense(units = UNITSOUT, activation = "softmax", name = "Dense_4")

x = dense1(inputs)
x = dense2(x)
x = dense3(x)
outputs = dense4(x)

model = Model(inputs = inputs, outputs = outputs)
model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
training_history = model.fit(X_train_scaled, y_train, epochs = EPOCHS, batch_size = BATCHS, validation_data=(X_test_scaled,y_test))

#calcul du score
score = model.evaluate(X_test_scaled, y_test)

#prédiction
test_pred = model.predict(X_test_scaled)

y_test_class = y_test
y_pred_class = np.argmax(test_pred, axis = 1)

#Résultats
precis = classification_report(y_test_class, y_pred_class,output_dict=True)

#Output
display(Markdown("""<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 3 couches denses :
<ul><li>la première avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la seconde avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la troisième avec <b>{} neurones</b> et une fonction d'activation <b>{}</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>{} epochs</b> par batch de <b>{}</b>.</li></ul>
""".format(UNITS1, ACTIV1, UNITS2, ACTIV2, UNITS3, ACTIV3, EPOCHS, BATCHS)))

model.summary()

display(Markdown("""<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>{} %</b> (<i>recall = <b>{} %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>{} %</b>.</li></ul>
""".format(
    round(100*precis["1"]["precision"],2),
    round(100*precis["1"]["recall"],2),
    round(100*precis["0"]["precision"],2),
    round(100*precis["0"]["recall"],2),
    round(100*precis["accuracy"],2)
)))

display(Markdown("""L'ajout d'une troisième couche de neurones ne semble pas modifier les performances.
"""))

display(Markdown("<i><h5>Rapport de classification :</h5></i>"))
print(classification_report(y_test_class, y_pred_class))
display(Markdown("<i><h5>Matrice de confusion :</h5></i>"))
print(pd.crosstab(y_test_class, y_pred_class, rownames = ["Classe réelle"], colnames = ["Classe prédite"]))

<h2><center><u>Effet du nombre de couches de neurones</u></center></h2>
<h4><u>Entrainement du modèle :</u></h4>

Epoch 1/6
3229/3229 [==============================] - 3s 979us/step - loss: 0.3541 - accuracy: 0.8467 - val_loss: 0.3398 - val_accuracy: 0.8536
Epoch 2/6
3229/3229 [==============================] - 3s 916us/step - loss: 0.3449 - accuracy: 0.8508 - val_loss: 0.3372 - val_accuracy: 0.8542
Epoch 3/6
3229/3229 [==============================] - 3s 918us/step - loss: 0.3425 - accuracy: 0.8513 - val_loss: 0.3397 - val_accuracy: 0.8548
Epoch 4/6
3229/3229 [==============================] - 3s 921us/step - loss: 0.3406 - accuracy: 0.8522 - val_loss: 0.3342 - val_accuracy: 0.8553
Epoch 5/6
3229/3229 [==============================] - 3s 918us/step - loss: 0.3392 - accuracy: 0.8527 - val_loss: 0.3335 - val_accuracy: 0.8552
Epoch 6/6
808/808 [==============================] - 1s 749us/step - loss: 0.3333 - accuracy: 0.8541


<h4><u>Récapitulatif du modèle :</u></h4>
Ce modèle avait 3 couches denses :
<ul><li>la première avec <b>25 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la seconde avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la troisième avec <b>50 neurones</b> et une fonction d'activation <b>relu</b> ;</li>
<li> la couche de sortie comportait <b>2 neurones</b> et une fonction d'activation <b>softmax</b> ;</li>
<li> apprentissage sur <b>6 epochs</b> par batch de <b>32</b>.</li></ul>


Model: "model_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 22)]              0         
                                                                 
 Dense_1 (Dense)             (None, 25)                575       
                                                                 
 Dense_2 (Dense)             (None, 50)                1300      
                                                                 
 Dense_3 (Dense)             (None, 50)                2550      
                                                                 
 Dense_4 (Dense)             (None, 2)                 102       
                                                                 
Total params: 4,527
Trainable params: 4,527
Non-trainable params: 0
_________________________________________________________________


<h4><u>Résultats du modèle :</u></h4>
<ul><li> Prédiction des <b>jours de pluie</b> avec une précision de <b>74.11 %</b> (<i>recall = <b>49.86 %</b></i>) ;</li>
<li> Prédiction des <b>jours sans pluie</b> avec une précision de <b>87.33 %</b> (<i>recall = <b>95.2 %</b></i>) ;</li>
<li>La précision globale du modèle est de <b>85.41 %</b>.</li></ul>


L'ajout d'une troisième couche de neurones ne semble pas modifier les performances.


<i><h5>Rapport de classification :</h5></i>

              precision    recall  f1-score   support

           0       0.87      0.95      0.91     20250
           1       0.74      0.50      0.60      5576

    accuracy                           0.85     25826
   macro avg       0.81      0.73      0.75     25826
weighted avg       0.84      0.85      0.84     25826



<i><h5>Matrice de confusion :</h5></i>

Classe prédite      0     1
Classe réelle              
0               19279   971
1                2796  2780
